In [2]:
#Habilitar intellisense
%config IPCompleter.greedy = True

## Arboles de decisión

In [10]:
import os
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv(os.path.join("diabetes.csv"))
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
feature_cols = ['Pregnancies', 'Insulin', 'BMI', 'Age','Glucose','BloodPressure','DiabetesPedigreeFunction']
X = df[feature_cols] # Features
y = df["Outcome"] # Target variable

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training, 30% test

## Optimización de parámetros

In [5]:
#https://github.com/conda-forge/hyperopt-feedstock
#conda install -c conda-forge hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK,Trials

In [6]:
space ={
    'n_estimators':hp.quniform('n_estimators',100,1000,1),  
    'learning_rate':hp.quniform('learning_rate',0.025,0.5,0.025),
    'max_depth':hp.quniform('max_depth',1,13,1),
    'subsample': hp.quniform('subsample',0.5,1,0.05),
    'colsample_bytree':hp.quniform('colsample_bytree',0.5,1,0.05),
    'nthread':6,
    'silent':1
}

In [7]:
def objective(params):
    params['n_estimators']=int(params['n_estimators'])
    params['max_depth']=int(params['max_depth'])  
    clf=XGBClassifier(**params)
    clf.fit(X_train,y_train)   
    accuracy = accuracy_score(y_test, clf.predict(X_test))
    return {'loss':1-accuracy,'status':STATUS_OK}

In [14]:
trials=Trials()

In [15]:
best=fmin(objective,space,algo=tpe.suggest,trials=trials,max_evals=20)

100%|██████████| 20/20 [00:07<00:00,  2.43it/s, best loss: 0.21212121212121215]


In [16]:
print(best)

{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.1, 'max_depth': 6.0, 'n_estimators': 185.0, 'subsample': 0.8}


In [17]:
best['n_estimators']=int(best['n_estimators'])
best['max_depth']=int(best['max_depth'])                      

In [18]:
tree_v5 = XGBClassifier(**best)
tree_v5

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9500000000000001, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=185, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [19]:
tree_v5.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9500000000000001, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=185, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [20]:
# métricas de desempeño
# accuracy
print('accuracy del clasificador - version 5 : {0:.2f}'.format(accuracy_score(y_test, tree_v5.predict(X_test))))
# confusion matrix
print('matriz de confusión del clasificador - version 5: \n {0}'.format(confusion_matrix(y_test, tree_v5.predict(X_test))))
# precision 
print('precision del clasificador - version 5 : {0:.2f}'.format(precision_score(y_test, tree_v5.predict(X_test))))
# precision 
print('recall del clasificador - version 5 : {0:.2f}'.format(recall_score(y_test, tree_v5.predict(X_test))))
# f1
print('f1 del clasificador - version 5 : {0:.2f}'.format(f1_score(y_test, tree_v5.predict(X_test))))

accuracy del clasificador - version 5 : 0.79
matriz de confusión del clasificador - version 5: 
 [[127  19]
 [ 30  55]]
precision del clasificador - version 5 : 0.74
recall del clasificador - version 5 : 0.65
f1 del clasificador - version 5 : 0.69


## Guardar el clasificador

Python cuenta con librerias de serialización que facilitan guardar el clasificador en un archivo (pickle, joblib); este archivo puede ser restaurado para hacer predicciones.

In [21]:
import pickle

In [22]:
#Cree la carpeta 'clasificador' en el folder donde está el notebook
ruta_archivo_clasificador = os.path.join("clasificador", "tree_v5.pkl")

In [23]:
#Abrir el archivo para escribir contenido binario
archivo_clasificador = open(ruta_archivo_clasificador, "wb")

In [24]:
200OK
Time:pickle.dump(tree_v5, archivo_clasificador)

In [25]:
archivo_clasificador.close()

## Cargar el clasificador

In [28]:
#Abrir el archivo en modo lectura de contenido binario y cargar el clasificdor
archivo_clasificador = open(ruta_archivo_clasificador, "rb")
tree_v6 = pickle.load(archivo_clasificador)
archivo_clasificador.close()

In [29]:
# métricas de desempeño
# accuracy
print('accuracy del clasificador - version 6 : {0:.2f}'.format(accuracy_score(y_test, tree_v6.predict(X_test))))
# confusion matrix
print('matriz de confusión del clasificador - version 6: \n {0}'.format(confusion_matrix(y_test, tree_v6.predict(X_test))))
# precision 
print('precision del clasificador - version 6 : {0:.2f}'.format(precision_score(y_test, tree_v6.predict(X_test))))
# precision 
print('recall del clasificador - version 6 : {0:.2f}'.format(recall_score(y_test, tree_v6.predict(X_test))))
# f1
print('f1 del clasificador - version 6 : {0:.2f}'.format(f1_score(y_test, tree_v6.predict(X_test))))

accuracy del clasificador - version 6 : 0.79
matriz de confusión del clasificador - version 6: 
 [[127  19]
 [ 30  55]]
precision del clasificador - version 6 : 0.74
recall del clasificador - version 6 : 0.65
f1 del clasificador - version 6 : 0.69


##  Modificar el clasificador

In [31]:
tree_v6.n_estimators = 200
tree_v6.n_estimators

200

In [32]:
tree_v6.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9500000000000001, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [33]:
ruta_archivo_clasificador = os.path.join("clasificador", "tree_v6.pkl")

In [34]:
archivo_clasificador = open(ruta_archivo_clasificador, "wb")

In [35]:
pickle.dump(tree_v6, archivo_clasificador)

In [36]:
archivo_clasificador.close()

In [37]:
archivo_clasificador = open(ruta_archivo_clasificador, "rb")
tree_v7 = pickle.load(archivo_clasificador)
archivo_clasificador.close()

In [38]:
tree_v7.n_estimators

200

##  Opciones de despliegue

<img src="35-opciones-despliegue.png" style="width:600px"/>

## Flask

Flask es un **framework** minimalista escrito en Python que permite crear aplicaciones web rápidamente y con un mínimo número de líneas de código - **Wikipedia**.

__[Flask](https://flask.palletsprojects.com/en/1.1.x/)__

In [ ]:
#python server.py